In [ ]:
import requests,random,re
import time
import os
import csv
import sys
import json
import importlib
from fake_useragent import UserAgent
from lxml import etree
from lxml import html
import requests
import re


In [ ]:
from selenium import webdriver
import time
from tqdm import tqdm
try:
    print(u'登陆新浪微博手机端...')
    browser = webdriver.Firefox()
    ##给定登陆的网址
    url = 'https://passport.weibo.cn/signin/login'
    browser.get(url)
    time.sleep(3)
    #找到输入用户名的地方
    username = browser.find_element_by_css_selector('#loginName')
    time.sleep(2)
    username.clear()
    username.send_keys('15528076813')#输入自己的账号
    #找到输入密码的地方
    password = browser.find_element_by_css_selector('#loginPassword')
    time.sleep(2)
    password.send_keys('wANG123456')
    #点击登录
    browser.find_element_by_css_selector('#loginAction').click()
    time.sleep(2)
except:
    print('########出现Error########')
finally:
    print('完成登陆!')

In [ ]:
#话题
topic='植物肉'

#是否只关注话题
focus_topic=True

#结果页数
page_number=11

In [ ]:
#只关注话题
if focus_topic:
    topic='%23'+topic+'%23'

# 获取微博话题下的内容
def get_page(i):
    url = 'https://s.weibo.com/weibo?q='+topic+'&page='+str(i) 
    browser.get(url)
    time.sleep(3)

#将微博折叠隐藏的内容展开
def show_all():
    num = len(browser.find_elements("partial link text",'展开'))   
    for i in range(num):
        browser.find_element("partial link text",'展开').click()
        time.sleep(1)

#提取微博文本内容
text = []
def get_info():
    info = browser.find_elements("xpath","//p[@class='txt']")
    for value in info: 
        text.append(value.text)

if page_number==1:
    url = 'https://s.weibo.com/weibo?q='+topic
    browser.get(url)
    time.sleep(3)
    show_all()
    get_info()
else:
    for i in tqdm(range(1,page_number)):
        get_page(i)
        show_all()
        get_info()

#删除空元素
while '' in text:
    text.remove('')


In [ ]:
# just for test

# print(len(text))
# text

In [ ]:
import codecs
import pandas as pd

file1 = './data/'+topic+'.txt'
file2 = './data/'+topic+'.csv'

with codecs.open(file1,'w+',encoding='utf-8') as f:
    f.write("")   
with codecs.open(file2,'w+',encoding='utf-8') as f:
    f.write("") 

with open('./data/'+topic+'.txt',"w",encoding='utf-8') as f:
    for i in range(len(text)):
        f.write(text[i])

pd.DataFrame(text).to_csv('./data/'+topic+'.csv',encoding='utf_8_sig',index=False)

文本清洗

In [ ]:
# 数据读取
filepath_p = './data/'+topic+'.csv'
data = pd.read_csv(filepath_p)
data.head()

theme = data
theme=theme.dropna()  ##删除缺失值
theme.shape


In [ ]:
# 特殊符号去除
import re
import warnings

warnings.filterwarnings("ignore")

r1 = "[^\u4e00-\u9fa5]"

theme['clean_text'] = ''
text_len = theme.shape[0]
for i in range(text_len):
    a = re.sub(r1,'',theme.iloc[i,0])
    theme.iloc[i,1] = a
theme.head()

theme.clean_text.to_csv("./data/clean_"+topic+".txt",index=False)
with open("./data/clean_"+topic+".txt",'r',encoding='UTF-8') as f:
    contents = f.read()


In [ ]:
import jieba

# 载入标准停用词库
stopwords1= [line.rstrip() for line in open('./stopwords/baidu_stopwords.txt', 'r', encoding='utf-8')]
stopwords2= [line.rstrip() for line in open('./stopwords/cn_stopwords.txt', 'r', encoding='utf-8')]
stopwords3 = [line.rstrip() for line in open('./stopwords/scu_stopwords.txt', 'r', encoding='utf-8')]
stopwords4 = [line.rstrip() for line in open('./stopwords/hit_stopwords.txt', 'r', encoding='utf-8')]


#自定义停用词
stopwords_mine=["收起","全文","视频","微博"]

stopwords = stopwords1 + stopwords2 + stopwords3+stopwords_mine

In [ ]:
#清洗停用词数据
file3="./data/words_"+topic+".txt"
with codecs.open(file3,'w+',encoding='utf-8') as f:
    f.write("")   

output = open(file3, "w", encoding='utf8')

# 使用jieba分词，获取词的列表
contents_cut = jieba.cut(contents)
for word in contents_cut:
    if  word not in stopwords:
        output.write(word + " ")

contents_list = " ".join(contents_cut)


In [ ]:
import jieba
file = open(file3, "r", encoding='utf-8') 
txt = file.read()

words = jieba.lcut(txt) # 使用jieba进行分词，将文本分成词语列表
count = {}

for word in words: 
    #纳入统计的词长度
    if len(word) > 1: 
        count[word] = count.get(word, 0) + 1 


# 删除无关词
# 自定义无关词语
exclude = ["可以", "一起", "这样","clean","text"] 
for key in list(count.keys()): 
    if key in exclude:
        del count[key] 

list_words = list(count.items()) 

list_words.sort(key=lambda x: x[1], reverse=True) 

for i in range(5): 
    word, number = list_words[i]
print("关键字：{:-<10}频次：{:+>8}".format(word, number))